In [1]:
import os
import numpy as np
import pandas as pd
import glob

from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split

from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

from sklearn.decomposition import TruncatedSVD

In [2]:
os.chdir(r'C:\Users\ashut\Downloads\AIML\7-Recommendation Systems\Week 3\Data Set')

In [3]:
all_files = glob.glob("*.csv")

In [4]:
# Reading all the csv file in one dataframe

smartph_data= pd.concat((pd.read_csv(f,encoding = "ISO-8859-1") for f in all_files), ignore_index=True)

In [5]:
smartph_data.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [6]:
smartph_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415133 entries, 0 to 1415132
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1415133 non-null  object 
 1   date       1415133 non-null  object 
 2   lang       1415133 non-null  object 
 3   country    1415133 non-null  object 
 4   source     1415133 non-null  object 
 5   domain     1415133 non-null  object 
 6   score      1351644 non-null  float64
 7   score_max  1351644 non-null  float64
 8   extract    1395772 non-null  object 
 9   author     1351931 non-null  object 
 10  product    1415132 non-null  object 
dtypes: float64(2), object(9)
memory usage: 118.8+ MB


In [7]:
smartph_data.shape

(1415133, 11)

In [8]:
# Rounding the score values to the nearest integer

smartph_data=smartph_data.round({'score':0})

In [9]:
smartph_data

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.0,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8
...,...,...,...,...,...,...,...,...,...,...,...
1415128,/cellphones/alcatel-ot-club_1187/,5/12/2000,de,de,Ciao,ciao.de,2.0,10.0,Weil mein Onkel bei ALcatel arbeitet habe ich ...,david.paul,Alcatel Club Plus Handy
1415129,/cellphones/alcatel-ot-club_1187/,5/11/2000,de,de,Ciao,ciao.de,10.0,10.0,Hy Liebe Leserinnen und Leser!! Ich habe seit ...,Christiane14,Alcatel Club Plus Handy
1415130,/cellphones/alcatel-ot-club_1187/,5/4/2000,de,de,Ciao,ciao.de,2.0,10.0,"Jetzt hat wohl Alcatell gedacht ,sie machen wa...",michaelawr,Alcatel Club Plus Handy
1415131,/cellphones/alcatel-ot-club_1187/,5/1/2000,de,de,Ciao,ciao.de,8.0,10.0,Ich bin seit 2 Jahren (stolzer) Besitzer eines...,claudia0815,Alcatel Club Plus Handy


In [10]:
# Identifing the null values

smartph_data.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64

In [11]:
# Replacing the null values with the mean

smartph_data['score'].fillna((smartph_data['score'].mean()),inplace=True)
smartph_data['score_max'].fillna((smartph_data['score_max'].mean()),inplace=True)
smartph_data['product'].fillna(np.nan, inplace=True)

In [12]:
# Removing the duplicate values

smartph_data.drop_duplicates(inplace=True)

In [13]:
# Selecting 1000000 data randomly

smartph_data_copy = smartph_data.sample(n=1000000,random_state=612)

In [14]:
smartph_data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 430175 to 92037
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1000000 non-null  object 
 1   date       1000000 non-null  object 
 2   lang       1000000 non-null  object 
 3   country    1000000 non-null  object 
 4   source     1000000 non-null  object 
 5   domain     1000000 non-null  object 
 6   score      1000000 non-null  float64
 7   score_max  1000000 non-null  float64
 8   extract    986449 non-null   object 
 9   author     956191 non-null   object 
 10  product    999999 non-null   object 
dtypes: float64(2), object(9)
memory usage: 91.6+ MB


In [15]:
smartph_data_copy.isna().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score            0
score_max        0
extract      13551
author       43809
product          1
dtype: int64

In [16]:
# Dropping the irrelevant columns

smartph_data_copy.drop(['phone_url','date','lang','country','source','domain','extract'],axis=1,inplace=True)

In [17]:
smartph_data_copy.dropna(inplace=True)

In [18]:
# Users respective to there number of ratings

smrtphn_df1=smartph_data_copy.groupby('author')['score'].count().sort_values(ascending=False)
smrtphn_df1

author
Amazon Customer        54542
Cliente Amazon         13661
e-bit                   5959
Client d'Amazon         5495
Amazon Kunde            3283
                       ...  
abishek kalaiselvan        1
abisheridee                1
abitawheat                 1
abitoy                     1
 efef                      1
Name: score, Length: 597173, dtype: int64

In [19]:
# No. of users who have given more than 50 reviews

print(sum(smrtphn_df1 >= 50))

660


In [20]:
smrtphn_df2 = smartph_data_copy.groupby('product')['score'].count().sort_values(ascending=False)
smrtphn_df2

product
Lenovo Vibe K4 Note (White,16GB)                                                                                              3709
Lenovo Vibe K4 Note (Black, 16GB)                                                                                             3083
OnePlus 3 (Graphite, 64 GB)                                                                                                   2890
OnePlus 3 (Soft Gold, 64 GB)                                                                                                  2521
Samsung Galaxy Express I8730                                                                                                  1895
                                                                                                                              ... 
Nexus 4 LG-E960 -16GB- Black (New Boxed, Unlocked) Smartphone Android 4.2                                                        1
Nexus 5                                                                    

In [21]:
# No. of products which has more than 50 reviews

print(sum(smrtphn_df2 >= 50))

4211


In [22]:
smrtphn_df3= smartph_data_copy.groupby("product").filter(lambda x:x['score'].count() >=50)

In [23]:
smrtphn_df3

,score,score_max,author,product
430175,8.0,10.0,Vinod Kumar Chengespur,"Lenovo Vibe K4 Note (White,16GB)"
587153,2.0,10.0,Sharon,HTC Desire 816 Black (Virgin mobile) - 5.5 inc...
636378,2.0,10.0,Jomine Jose,Lava Iris X5 (Black)
230845,8.0,10.0,Cliente Amazon,"Microsoft Telefonia Lumia 950 XL Smartphone, 3..."
1290191,8.0,10.0,Hjkage,Nokia 5610 XpressMusic
...,...,...,...,...
43605,10.0,10.0,LeeAB,Apple iPhone 6s
954684,10.0,10.0,LuisMu,Sony Xperia Tipo - Smartphone libre (pantalla ...
1053936,10.0,10.0,loca14325,HTC Touch HD2
1002309,10.0,10.0,Monalisa,Samsung Galaxy Y


In [24]:
smrtphn_df3=smrtphn_df3.groupby("author").filter(lambda x:x['score'].count() >=50)

In [25]:
smrtphn_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90101 entries, 230845 to 243653
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   score      90101 non-null  float64
 1   score_max  90101 non-null  float64
 2   author     90101 non-null  object 
 3   product    90101 non-null  object 
dtypes: float64(2), object(2)
memory usage: 3.4+ MB


In [26]:
smrtphn_df3.describe()

,score,score_max
count,90101.000000,90101.0
mean,7.781238,10.0
std,2.808221,0.0
min,1.000000,10.0
25%,6.000000,10.0
50%,8.008083,10.0
75%,10.000000,10.0
max,10.000000,10.0


In [27]:
smrtphn_df3.drop(['score_max'],axis=1,inplace=True)

###### Popularity based for top 5 phones

In [28]:
ratings_mean_count = pd.DataFrame(smartph_data_copy.groupby('product')['score'].mean())

ratings_mean_count['score_counts'] = pd.DataFrame(smartph_data_copy.groupby('product')['score'].count()) 

In [29]:
ratings_mean_count.head()

,score,score_counts
product,,
"'Sony Xperia X (F5122) â White â Dual Sim (Google Android 6.0.1, 5 Display, 2 x CORTEX A72 1.8 GHz + 4 x cortex-a53...",10.000000,1
"'Sony Xperia X (F5122) â rosa â Dual Sim (Google Android 6.0.1, 5 Display, 2 x CORTEX A72 1.8 GHz + 4 x cortex-a53...",10.000000,1
"(CUBOT) GT88 5.5"" qHD 1.3GHz MTK6572 2-Core Android 4.2.2 3G Phone 8MP CAM 512MB RAM 4GB ROM",8.000000,1
"(DG300 Versione Aggiornata)5'' DOOGEE VOYAGER2 DG310 Dual Flashlights IPS Screen 3G Smartphone Android 4.4 MTK6582 1.3GHz Quad Core Telefono Cellulare Dual SIM 8G ROM OTG OTA GPS WIFI, BIANCO",7.388889,36
(Part 2) Lenovo VIBE X2,8.008083,1


In [30]:
smrtphn_df3=smrtphn_df3[['author','product','score']]

In [31]:
smrtphn_df3

,author,product,score
230845,Cliente Amazon,"Microsoft Telefonia Lumia 950 XL Smartphone, 3...",8.0
505343,Amazon Customer,Nokia Lumia 635 8GB Unlocked GSM 4G LTE Window...,2.0
766631,Amazon Customer,"Lenovo Used Lenovo Zuk Z1 (Space Grey, 64GB)",2.0
1244089,Anonymous,Samsung Rant,10.0
119096,Amazon Customer,"OnePlus 3T (Gunmetal, 6GB RAM + 64GB memory)",10.0
...,...,...,...
463857,Amazon Kunde,"Samsung Galaxy S5 mini Smartphone (4,5 Zoll (1...",6.0
1076134,e-bit,Celular Motorola EX115,6.0
161619,ÐÐ¼Ð¸ÑÑÐ¸Ð¹,Samsung Galaxy A3 (2016),8.0
499648,Amazon Customer,"Lenovo PHAB Plus Tablet (6.8 inch, 32GB, Wi-Fi...",2.0


######  collaborative filtering model using kNNWithMeans

In [32]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(smrtphn_df3,reader)

In [33]:
trainset, testset = train_test_split(data, test_size=0.3,random_state=10)

In [34]:
algo = KNNWithMeans(k=5, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [35]:
test_pred = algo.test(testset)

In [36]:
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Item-based Model : Test Set
RMSE: 3.9929


3.9928735985898665

In [37]:
new_data=smrtphn_df3.head(5000)
ratings_matrix = new_data.pivot_table(values='score', index='author', columns='product', fill_value=0)

In [38]:
ratings_matrix.shape

(270, 1703)

In [39]:
X = ratings_matrix.T
X.head()

author,#,????????,??????????,????????????,??????????????,?????????????? ??????????????,?????????????????,???????????,Aksisey,Alan,...,ÐÐ»ÐµÐ³,ÐÐ»ÑÐ³Ð°,ÐÐ°Ð²ÐµÐ»,Ð Ð¾Ð¼Ð°Ð½,Ð ÑÑÐ»Ð°Ð½,Ð¡Ð²ÐµÑÐ»Ð°Ð½Ð°,Ð¡ÐµÑÐ³ÐµÐ¹,Ð¢Ð°ÑÑÑÐ½Ð°,Ð®Ð»Ð¸Ñ,Ð®ÑÐ¸Ð¹
product,,,,,,,,,,,,,,,,,,,,,
"AICEK Coque ASUS ZenFone 3 ZE520KL, AICEK Etui Silicone Gel ASUS ZenFone 3 Housse Antichoc ZenFone 3 Transparente Souple Coque de Protection pour ASUS ZenFone 3(5.2 Pouces)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"AICEK Coque Samsung Galaxy A3 2016, AICEK Etui Silicone Gel Samsung Galaxy A3 2016 (A310F) Housse Antichoc Samsung A3 Transparente Souple Coque De Protection Pour Samsung Galaxy A3 2016 (4,7 pouces)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"APPLE iPhone 6s Plus - 128 GB, Rose Gold",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"APPLE iPhone 7 - Silver, 128 GB",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"APPLE iPhone 7 - Silver, 32 GB",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
X1 = X

X.shape

(1703, 270)

In [41]:
#Decomposing the Matrix

SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(1703, 10)

In [42]:
#Correlation Matrix

correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(1703, 1703)

In [43]:
product_names = list(X.index)
product_ID = product_names.index(X.index[1702])
product_ID

1702

In [44]:
#Correlation for all items with the item purchased by this customer based on items rated by other customers people who bought the same product

correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(1703,)

###### Top 5 products for test users.

In [45]:
#Recommending top 5 highly correlated products in sequence

Recommend = list(X.index[correlation_product_ID > 0.65])

# Removes the item already bought by the customer
Recommend.remove(X.index[1702]) 

Recommend[0:5]

['ASUS ZenFone 3 Max ZC520TL 16GB (Ñ\x81ÐµÑ\x80ÐµÐ±Ñ\x80Ð¸Ñ\x81Ñ\x82Ñ\x8bÐ¹)',
 'Alcatel One Touch Idol 3 Smartphone,4.7", Single SIM, Grigio Scuro [Italia]',
 'Apple Apple iPhone 5s - 32GB - Zilver',
 'Apple iPhone 3GS 32GB',
 'Apple iPhone 6s Plus Unlocked GSM 4G LTE Smartphone with 12MP Camera, 16 GB (Rose Gold)']

###### In what business scenario you should use popularity based Recommendation Systems ?

Popularity based recommendation system works with the trend(what most of the population likes). It basically uses the items which are in trend. For example, if any product which is usually bought by every new user then there are chances that it may suggest that item to the user who just signed up. Example: IMDB movie recommendation

The Popularity based recommender provide a general chart of recommended to all the users. They are not sensitive to the interests and tastes of a particular user.

###### In what business scenario you should use CF based Recommendation Systems ?

CF recommendation system is used for making automatic predictions about the interests of a user by collecting preferences or taste information from many users i.e. collaboration. These predictions are specific to the user, but use information gained from multiple users. This differs from the simpler approach of giving an average (non-specific) score for each item of interest, for example based on its number of votes(popularity based).

So, the business scenarios where user specific recommendation is required, we make used of different form of CF rommendation system.

###### What other possible methods can you think of which can further improve the recommendation for different users ?

The other methods which can be used for recommendations are:
1) Hybrid filtering
2) Evaluation metrics